In [1]:
import pandas as pd
import requests
import sys
import json
import numpy as np

In [2]:
df = pd.read_csv('fire_and_weather_data-Copy1.csv', low_memory=False)
len(df)

250342

In [3]:
df.columns

Index(['Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0', 'X', 'Y', 'FireCause',
       'FireDiscoveryDateTime', 'POOCounty', 'POOFips', 'POOState',
       'weather_data'],
      dtype='object')

In [4]:
df.drop(df.filter(regex="Unnamed"),axis=1, inplace=True)

In [5]:
df.head()

,X,Y,FireCause,FireDiscoveryDateTime,POOCounty,POOFips,POOState,weather_data
0,-118.180712,33.808985,Unknown,2020/02/28 20:45:40+00,Los Angeles,06037,US-CA,"{""queryCost"":1,""latitude"":33.8089848330001,""lo..."
1,-117.153901,33.176394,NaN,2019/07/01 19:54:00+00,San Diego,06073,US-CA,"{""queryCost"":1,""latitude"":33.1763937050001,""lo..."
2,-121.104180,38.834727,NaN,2016/06/20 22:05:59+00,Placer,06061,US-CA,"{""queryCost"":1,""latitude"":38.8347273530001,""lo..."
3,-117.228592,33.782442,Undetermined,2021/11/25 15:17:32+00,Riverside,06065,US-CA,"{""queryCost"":1,""latitude"":33.7824418820001,""lo..."
4,-118.309032,33.941815,Undetermined,2022/11/21 11:25:33+00,Los Angeles,06037,US-CA,"{""queryCost"":1,""latitude"":33.9418148400001,""lo..."


In [6]:
df['weather_data'].isna().sum()

0

In [7]:
wd = json.loads(df.at[150000, 'weather_data'])

In [17]:
wd

{'datetime': '2019-07-01',
 'datetimeEpoch': 1561964400,
 'tempmax': 74.6,
 'tempmin': 61.9,
 'temp': 67.5,
 'feelslikemax': 74.6,
 'feelslikemin': 61.9,
 'feelslike': 67.5,
 'dew': 59.6,
 'humidity': 76.9,
 'precip': 0.0,
 'precipprob': 0.0,
 'precipcover': 0.0,
 'preciptype': None,
 'snow': 0.0,
 'snowdepth': 0.0,
 'windgust': 18.3,
 'windspeed': 10.2,
 'winddir': 247.0,
 'pressure': 1013.8,
 'cloudcover': 35.9,
 'visibility': 6.2,
 'solarradiation': 535.6,
 'solarenergy': 28.8,
 'uvindex': 10.0,
 'sunrise': '05:43:13',
 'sunriseEpoch': 1561984993,
 'sunset': '20:01:40',
 'sunsetEpoch': 1562036500,
 'moonphase': 0.97,
 'conditions': 'Partially cloudy',
 'description': 'Clearing in the afternoon.',
 'icon': 'partly-cloudy-day',
 'stations': ['KCRQ',
  'KNFG',
  'D1175',
  '72293453121',
  'KOKB',
  'KL18',
  '72292703177'],
 'source': 'obs'}

In [9]:
wd['days'][0]['humidity']

54.2

In [10]:
df[['temp', 'humidity', 'precip', 'windspeed', 'winddir', 'solarradiation']] = ''

In [11]:
df.head()

,X,Y,FireCause,FireDiscoveryDateTime,POOCounty,POOFips,POOState,weather_data,temp,humidity,precip,windspeed,winddir,solarradiation
0,-118.180712,33.808985,Unknown,2020/02/28 20:45:40+00,Los Angeles,06037,US-CA,"{""queryCost"":1,""latitude"":33.8089848330001,""lo...",,,,,,
1,-117.153901,33.176394,NaN,2019/07/01 19:54:00+00,San Diego,06073,US-CA,"{""queryCost"":1,""latitude"":33.1763937050001,""lo...",,,,,,
2,-121.104180,38.834727,NaN,2016/06/20 22:05:59+00,Placer,06061,US-CA,"{""queryCost"":1,""latitude"":38.8347273530001,""lo...",,,,,,
3,-117.228592,33.782442,Undetermined,2021/11/25 15:17:32+00,Riverside,06065,US-CA,"{""queryCost"":1,""latitude"":33.7824418820001,""lo...",,,,,,
4,-118.309032,33.941815,Undetermined,2022/11/21 11:25:33+00,Los Angeles,06037,US-CA,"{""queryCost"":1,""latitude"":33.9418148400001,""lo...",,,,,,


In [12]:
df.tail(5)

,X,Y,FireCause,FireDiscoveryDateTime,POOCounty,POOFips,POOState,weather_data,temp,humidity,precip,windspeed,winddir,solarradiation
250337,-95.081657,35.411146,Human,2023/02/28 23:35:00+00,Haskell,40061,US-OK,"{""queryCost"":1,""latitude"":35.411146306,""longit...",,,,,,
250338,-95.288896,33.190561,Undetermined,2023/02/28 22:41:00+00,Franklin,48159,US-TX,"{""queryCost"":1,""latitude"":33.1905614400001,""lo...",,,,,,
250339,-95.278237,31.444536,Undetermined,2023/02/28 22:47:00+00,Houston,48225,US-TX,"{""queryCost"":1,""latitude"":31.4445355400001,""lo...",,,,,,
250340,-89.436166,37.657667,Human,2023/02/28 22:33:12+00,Jackson,17077,US-IL,"{""queryCost"":1,""latitude"":37.657667002,""longit...",,,,,,
250341,-97.455858,34.225956,Human,2023/02/28 18:59:59+00,Carter,40019,US-OK,"{""queryCost"":1,""latitude"":34.2259559760001,""lo...",,,,,,


In [29]:
for index, row in df.iterrows():
    if index%500 == 0:
        print('\r',index,end='',flush=True)
    if pd.isnull(row["weather_data"]):
        break
    wd = json.loads(row['weather_data'])
    wd = wd['days'][0]
    df.at[index, 'temp'] = wd['temp'] if 'temp' in wd else np.nan
    df.at[index, 'humidity'] = wd['humidity'] if 'humidity' in wd else np.nan
    df.at[index, 'precip'] = wd['precip'] if 'precip' in wd else np.nan
    df.at[index, 'windspeed'] = wd['windspeed'] if 'windspeed' in wd else np.nan
    df.at[index, 'winddir'] = wd['winddir'] if 'winddir' in wd else np.nan
    df.at[index, 'solarradiation'] = wd['solarradiation'] if 'solarradiation' in wd else np.nan
    
df.to_csv("combined_fire_and_weather_data.csv")

 250000

In [30]:
df

,X,Y,FireCause,FireDiscoveryDateTime,POOCounty,POOFips,POOState,weather_data,temp,humidity,precip,windspeed,winddir,solarradiation
0,-118.180712,33.808985,Unknown,2020/02/28 20:45:40+00,Los Angeles,06037,US-CA,"{""queryCost"":1,""latitude"":33.8089848330001,""lo...",68.5,32.7,0.0,9.1,212.1,335.8
1,-117.153901,33.176394,NaN,2019/07/01 19:54:00+00,San Diego,06073,US-CA,"{""queryCost"":1,""latitude"":33.1763937050001,""lo...",67.5,76.9,0.0,10.2,247.0,535.6
2,-121.104180,38.834727,NaN,2016/06/20 22:05:59+00,Placer,06061,US-CA,"{""queryCost"":1,""latitude"":38.8347273530001,""lo...",77.0,31.9,0.0,7.2,291.5,351.7
3,-117.228592,33.782442,Undetermined,2021/11/25 15:17:32+00,Riverside,06065,US-CA,"{""queryCost"":1,""latitude"":33.7824418820001,""lo...",59.7,9.7,0.0,21.8,29.6,304.2
4,-118.309032,33.941815,Undetermined,2022/11/21 11:25:33+00,Los Angeles,06037,US-CA,"{""queryCost"":1,""latitude"":33.9418148400001,""lo...",63.7,25.9,0.0,8.9,314.9,65.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250337,-95.081657,35.411146,Human,2023/02/28 23:35:00+00,Haskell,40061,US-OK,"{""queryCost"":1,""latitude"":35.411146306,""longit...",60.1,46.3,0.0,16.8,216.6,229.3
250338,-95.288896,33.190561,Undetermined,2023/02/28 22:41:00+00,Franklin,48159,US-TX,"{""queryCost"":1,""latitude"":33.1905614400001,""lo...",64.1,43.8,0.0,10.5,212.8,171.9
250339,-95.278237,31.444536,Undetermined,2023/02/28 22:47:00+00,Houston,48225,US-TX,"{""queryCost"":1,""latitude"":31.4445355400001,""lo...",63.2,70.2,0.0,10.3,188.9,174.7
250340,-89.436166,37.657667,Human,2023/02/28 22:33:12+00,Jackson,17077,US-IL,"{""queryCost"":1,""latitude"":37.657667002,""longit...",54.0,66.9,0.0,12.7,68.1,198.6


In [36]:
df['solarradiation'].isna().sum()

52523

In [37]:
len(df[~df.isnull().any(axis=1)])

179436

In [38]:
weather_df = df[~df.isnull().any(axis=1)]

In [39]:
weather_df.to_csv("cleaned_wildfire_data.csv")

In [40]:
weather_df

,X,Y,FireCause,FireDiscoveryDateTime,POOCounty,POOFips,POOState,weather_data,temp,humidity,precip,windspeed,winddir,solarradiation
0,-118.180712,33.808985,Unknown,2020/02/28 20:45:40+00,Los Angeles,06037,US-CA,"{""queryCost"":1,""latitude"":33.8089848330001,""lo...",68.5,32.7,0.0,9.1,212.1,335.8
3,-117.228592,33.782442,Undetermined,2021/11/25 15:17:32+00,Riverside,06065,US-CA,"{""queryCost"":1,""latitude"":33.7824418820001,""lo...",59.7,9.7,0.0,21.8,29.6,304.2
4,-118.309032,33.941815,Undetermined,2022/11/21 11:25:33+00,Los Angeles,06037,US-CA,"{""queryCost"":1,""latitude"":33.9418148400001,""lo...",63.7,25.9,0.0,8.9,314.9,65.3
8,-108.270562,47.003940,Human,2019/05/02 21:32:49+00,Petroleum,30069,US-MT,"{""queryCost"":1,""latitude"":47.0039400110001,""lo...",41.9,51.6,0.0,22.0,280.6,275.6
10,-115.946014,48.779806,Unknown,2018/05/22 14:21:50+00,Lincoln,30053,US-MT,"{""queryCost"":1,""latitude"":48.779806384,""longit...",65.5,51.6,0.0,7.0,339.9,324.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250337,-95.081657,35.411146,Human,2023/02/28 23:35:00+00,Haskell,40061,US-OK,"{""queryCost"":1,""latitude"":35.411146306,""longit...",60.1,46.3,0.0,16.8,216.6,229.3
250338,-95.288896,33.190561,Undetermined,2023/02/28 22:41:00+00,Franklin,48159,US-TX,"{""queryCost"":1,""latitude"":33.1905614400001,""lo...",64.1,43.8,0.0,10.5,212.8,171.9
250339,-95.278237,31.444536,Undetermined,2023/02/28 22:47:00+00,Houston,48225,US-TX,"{""queryCost"":1,""latitude"":31.4445355400001,""lo...",63.2,70.2,0.0,10.3,188.9,174.7
250340,-89.436166,37.657667,Human,2023/02/28 22:33:12+00,Jackson,17077,US-IL,"{""queryCost"":1,""latitude"":37.657667002,""longit...",54.0,66.9,0.0,12.7,68.1,198.6
